In [2]:
from threading import Thread
from math import pi

import rclpy
from rclpy.callback_groups import ReentrantCallbackGroup
from rclpy.node import Node

from pymoveit2 import MoveIt2
from pymoveit2.robots import ur

from geometry_msgs.msg import Pose, Quaternion, Point
import rclpy.wait_for_message
from ur_msgs.srv import SetIO
from ur_commander.srv import VisualizePoses

rclpy.init()

# Create node for this example
node = Node("notebook_example")
callback_group = ReentrantCallbackGroup()


def display_poses(poses: list[Pose], frame_id: str = "ur_base_link") -> None:
    client = node.create_client(VisualizePoses, "/visualize_poses")
    while not client.wait_for_service(timeout_sec=1.0):
        node.get_logger().info("service not available, waiting again...")
    client.call(VisualizePoses.Request(poses=poses, frame_id=frame_id))


moveit2 = MoveIt2(
    node=node,
    joint_names=ur.joint_names(),
    base_link_name=ur.base_link_name(),
    end_effector_name=ur.end_effector_name(),
    group_name=ur.MOVE_GROUP_ARM,
    callback_group=callback_group,
)

# Spin the node in background thread(s) and wait a bit for initialization
executor = rclpy.executors.MultiThreadedExecutor(2)
executor.add_node(node)
executor_thread = Thread(target=executor.spin, daemon=True, args=())
executor_thread.start()
node.create_rate(1.0).sleep()

# Scale down velocity and acceleration of joints (percentage of maximum)
moveit2.max_velocity = 0.1
moveit2.max_acceleration = 0.05
synchronous = True
cancel_after_secs = 0.0
cartesian = False
cartesian_max_step = 0.0025
cartesian_fraction_threshold = 0.0
cartesian_jump_threshold = 0.0
cartesian_avoid_collisions = False
moveit2.planner_id = "RRTconnectkConfigDefault"


In [3]:
# Setting up collision boxes
husky_size = {"x":0.99, "y":0.67, "z":0.39}
boxes = {"x":0.325, "y":0.49, "z":0.40}
ground_height = -husky_size["z"] - 0.01/2 - 0.05

print(boxes["x"])

# Add collision objects
moveit2.add_collision_box(
    id="husky",
    size=[husky_size["x"] + 0.05, husky_size["y"] + 0.05, husky_size["z"] + 0.05],
    position=[-0.3, 0.0, -husky_size["z"]/2 - 0.05],
    quat_xyzw=[0.0, 0.0, 0.0, 0.0],
)

moveit2.add_collision_box(
    id="controller",
    size=[0.45, 0.65, 0.7],
    position=[-0.7, 0.0, 0.34],
    quat_xyzw=[0.0, 0.0, 0.0, 0.0],
)

moveit2.add_collision_box(
    id="floor",
    size=[4.0, 4.0, 0.01],
    position=[0.0, 0.0, -husky_size["z"] - 0.01/2 - 0.05],
    quat_xyzw=[0.0, 0.0, 0.0, 0.0],
)

0.325


**Define Joint Position**   

Forward kinematics, a trajectory will be planned

In [3]:
# plan home position
home_position = [0.0, -pi / 2, pi / 2, 0.0, 0.0, 0.0]
node.get_logger().info("Moving to home position")
traj = moveit2.move_to_configuration(home_position)
if traj is None:
    node.get_logger().error("Failed to move to home position")

[INFO] [1741870001.301883614] [notebook_example]: Moving to home position


In [4]:
# Excecute the planned trajectory home position
moveit2.execute(traj)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to execute trajectory")
else:
    node.get_logger().info("Moved to home position")

[INFO] [1741870011.267694773] [notebook_example]: Moved to home position


**Define a Target Pose**    

Display the pose

In [18]:
target = Pose(
    position=Point(x=.5, y=0.0, z=-0.1),
    orientation=Quaternion(x=0.0, y=90.0, z=90.0, w=0.7071067811865476),
)

display_poses([target])

In [20]:
from scipy.spatial.transform import Rotation as R
import math

# Define Euler angles in radians (roll, pitch, yaw)
roll = 0.0
pitch = math.radians(90)
yaw = math.radians(90)

# Convert Euler angles to quaternion
r = R.from_euler('xyz', [roll, pitch, yaw])
q = r.as_quat()  # Returns quaternion as [x, y, z, w]

# Update target orientation with the calculated quaternion
target.orientation.x = q[0]
target.orientation.y = q[1]
target.orientation.z = q[2]
target.orientation.w = q[3]



Plan a trajectory

In [21]:
position = [target.position.x, target.position.y, target.position.z]
quat_xyzw = [target.orientation.x, target.orientation.y, target.orientation.z, target.orientation.w]


node.get_logger().info(f"Moving to {{position: {list(position)}, quat_xyzw: {list(quat_xyzw)}}}")
traj = moveit2.plan(
    position=position,
    quat_xyzw=quat_xyzw,
    cartesian=True,
    cartesian_fraction_threshold=cartesian_fraction_threshold,
)

[INFO] [1741869629.648939316] [notebook_example]: Moving to {position: [0.5, 0.0, -0.1], quat_xyzw: [-0.4999999999999999, 0.5, 0.5, 0.5000000000000001]}


Execute the trajectory

In [22]:
moveit2.execute(traj)
success = moveit2.wait_until_executed()
if not success:
    node.get_logger().error("Failed to execute trajectory")

#### More than one target pose

In [7]:
target0 = Pose(
    position=Point(x=0.6, y=-0.3, z=0.5),
    orientation=Quaternion(x=0.0, y=0.7071067811865475, z=0.0, w=0.7071067811865476),
)
target1 = Pose(
    position=Point(x=0.6, y=0.3, z=0.5),
    orientation=Quaternion(x=0.5, y=0.5, z=0.5, w=0.5),
)

display_poses([target0, target1])

In [ ]:
targets = [target0, target1]
for i, target_pose in enumerate(targets):

    position = [target_pose.position.x, target_pose.position.y, target_pose.position.z]
    quat_xyzw = [
        target_pose.orientation.x,
        target_pose.orientation.y,
        target_pose.orientation.z,
        target_pose.orientation.w,
    ]

    node.get_logger().info(f"Moving to target {i} {{position: {list(position)}, quat_xyzw: {list(quat_xyzw)}}}")
    traj = moveit2.move_to_pose(
        position=position,
        quat_xyzw=quat_xyzw,
        cartesian=cartesian,
        cartesian_max_step=cartesian_max_step,
        cartesian_fraction_threshold=cartesian_fraction_threshold,
    )
    if traj is not None:
        node.get_logger().info(f"Move to target {i}")
        moveit2.execute(traj)
        success = moveit2.wait_until_executed()

[INFO] [1741860625.902908233] [notebook_example]: Moving to target 0 {position: [0.6, -0.3, 0.5], quat_xyzw: [0.0, 0.7071067811865475, 0.0, 0.7071067811865476]}
[INFO] [1741860636.008925550] [notebook_example]: Move to target 0


In [14]:
target_poses = [
    Pose(
        position=Point(x=0.6, y=0.1, z=0.8),
        orientation=Quaternion(x=0.0, y=0.7071067811865475, z=0.0, w=0.7071067811865476),
    ),
    Pose(
        position=Point(x=0.6, y=0.3, z=0.5),
        orientation=Quaternion(x=0.5, y=0.5, z=0.5, w=0.5),
    ),
    Pose(
        position=Point(x=0.6, y=-0.3, z=0.5),
        orientation=Quaternion(x=0.0, y=0.7071067811865475, z=0.0, w=0.7071067811865476),
    ),
]

display_poses(target_poses)

In [ ]:
traj = moveit2.move_sequence_async(
    poses=target_poses,
    blend_radius=[0.1, 0.1, 0.0],
    pipeline_id="pilz_industrial_motion_planner",
    planner_ids=["PTP", "PTP", "PTP"],
)

In [16]:
if traj is not None:
    moveit2.execute(traj)